In [24]:
import os

if not os.path.exists("boptestGymService"):
    !git clone -b boptest-gym-service https://github.com/ibpsa/project1-boptest-gym.git boptestGymService
else:
    print("Folder 'boptestGymService' already exists. Skipping clone.")

import sys
sys.path.insert(0,'boptestGymService')
from boptestGymEnv import BoptestGymEnv

import torch

torch.cuda.is_available()

Folder 'boptestGymService' already exists. Skipping clone.


True

In [36]:
env = BoptestGymEnv(url                  = 'http://localhost',
                    testcase             = 'bestest_hydronic_heat_pump',
                    actions              = ['oveHeaPumY_u'],
                    observations         = {'time':(0,604800),
                                            'reaTZon_y':(280.,310.),
                                            'TDryBul':(265,303),
                                            'HDirNor':(0,862),
                                            'InternalGainsRad[1]':(0,219),
                                            'PriceElectricPowerHighlyDynamic':(-0.4,0.4),
                                            'LowerSetp[1]':(280.,310.),
                                            'UpperSetp[1]':(280.,310.)},
                    predictive_period    = 24*3600,
                    regressive_period    = 6*3600,
                    random_start_time    = True,
                    max_episode_length   = 24*3600,
                    warmup_period        = 24*3600,
                    step_period          = 3600)

/home/ferran/projects/carles-mateu/a2/.venv/lib/python3.10/site-packages/gymnasium/spaces/box.py:130: UserWarning: WARN: Box bound precision lowered by casting to float32
  gym.logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [37]:
from boptestGymEnv import BoptestGymEnv, NormalizedObservationWrapper, DiscretizedActionWrapper
from stable_baselines3 import DQN

env = NormalizedObservationWrapper(env)
env = DiscretizedActionWrapper(env,n_bins_act=10)

In [43]:
model = DQN('MlpPolicy', env, verbose=1, gamma=0.99,
            learning_rate=5e-4, batch_size=1024, seed=123456,
            buffer_size=365*24, learning_starts=24, train_freq=1, device='cpu')

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [44]:
model.learn(total_timesteps=100)

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 24       |
|    ep_rew_mean     | -10      |
| time/              |          |
|    fps             | 4        |
|    iterations      | 1        |
|    time_elapsed    | 5        |
|    total_timesteps | 24       |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 24           |
|    ep_rew_mean          | -5.08        |
| time/                   |              |
|    fps                  | 4            |
|    iterations           | 2            |
|    time_elapsed         | 9            |
|    total_timesteps      | 48           |
| train/                  |              |
|    approx_kl            | 0.0066914237 |
|    clip_fraction        | 0.00417      |
|    clip_range           | 0.2          |
|    entropy_loss         | -2.4         |
|    explained_variance   | 0.0111       |
|    learning_r

In [18]:
# Loop for one episode of experience (one day as set in max_episode_length)
done = False
obs, _ = env.reset()
while not done:
  action, _ = model.predict(obs, deterministic=True)
  obs,reward,terminated,truncated,info = env.step(action)
  done = (terminated or truncated)

# Obtain KPIs for evaluation
env.get_kpis()

{'tdis_tot': 0.15685426466607122,
 'idis_tot': 0,
 'ener_tot': 0.2784581349563481,
 'cost_tot': 0.07058913721143426,
 'emis_tot': 0.04650250853771014,
 'pele_tot': 0.019584782613200152,
 'pgas_tot': None,
 'pdih_tot': None,
 'time_rat': 0.00023476835754182603}

In [33]:
env.stop()